In [1]:
import torch
import os
import glob as glob
import matplotlib.pyplot as plt
import pandas as pd
import json
import ultralytics
from PIL import Image
import shutil
import numpy as np
import yaml

In [2]:
batch_size = 16
resize_to = 640
num_epochs = 50

train_data = r'../ML/MLtest_dataset/train/'
valid_data = r'../ML/MLtest_dataset/val/'
test_data = r'../ML/MLtest_dataset/test/'

class_dict = { 0: "Dense", 1: "Diffuse", 2: "Mixed"}

Составление датасета в виде pandas.Dataframe из JSON формата

In [3]:
list_of_json_files_train = sorted([pos_json for pos_json in os.listdir(train_data) if pos_json.endswith('.json')])
list_of_json_files_train

['MLtest_json (10).json',
 'MLtest_json (11).json',
 'MLtest_json (2-5).json',
 'MLtest_json (6-7).json']

In [4]:
list_of_json_files_val = sorted([pos_json for pos_json in os.listdir(valid_data) if pos_json.endswith('.json')])
list_of_json_files_val

['MLtest_json (12).json', 'MLtest_json (13).json']

In [5]:
list_of_json_files_test = sorted([pos_json for pos_json in os.listdir(test_data) if pos_json.endswith('.json')])
list_of_json_files_test

['MLtest_json (81-9).json']

In [6]:
def df_from_json(dir_path: str, json_filenames_list: list) -> pd.DataFrame:
    res_df = pd.DataFrame()
    for json_file in json_filenames_list:
        f = open(f"{dir_path}{json_file}", 'r', encoding='utf-8')
        file = json.loads(f.read())

        for key in file.keys():
            df = pd.json_normalize(file[f"{key}"], record_path=['regions'], meta = ['filename'])
            if df.empty:
                continue
            else:
                res_df = pd.concat([res_df, df], ignore_index=False)
    res_df = res_df.drop_duplicates()
    res_df.reset_index(inplace=True)
    res_df.drop(['index', 'shape_attributes.name'], axis = 1, inplace=True)
    res_df.rename(columns={"shape_attributes.x": "xmin", "shape_attributes.y": "ymin", "shape_attributes.width": "width",
                           "shape_attributes.height": "height", "region_attributes.Type": "Type"}, errors="raise", inplace=True)
    res_df['img_width'] = 0
    res_df['img_height'] = 0
    all_filenames = set(res_df['filename'].tolist())
    for file in all_filenames:
        img = Image.open(f"{dir_path}{file}")
        width, height = img.size
        res_df.loc[res_df['filename'] == file, 'img_width'] = width
        res_df.loc[res_df['filename'] == file, 'img_height'] = height

    res_df = res_df[['filename', 'img_width', 'img_height', 'xmin', 'ymin', 'width', 'height', 'Type']]
    
    if res_df['Type'].isna().sum() > 0:
        res_df['Type'].fillna(value=res_df['Type'].mode()[0], inplace=True)
    return res_df

In [7]:
train_df = df_from_json(dir_path=train_data, json_filenames_list=list_of_json_files_train)
train_df

,filename,img_width,img_height,xmin,ymin,width,height,Type
0,чашка 10.jpg,4299,8498,1074,695,189,245,Dense
1,чашка 10.jpg,4299,8498,1634,537,284,150,Dense
2,чашка 10.jpg,4299,8498,2179,229,268,213,Dense
3,чашка 10.jpg,4299,8498,2779,308,174,166,Dense
4,чашка 10.jpg,4299,8498,3513,876,245,332,Dense
...,...,...,...,...,...,...,...,...
994,чашка 7.jpg,4363,8414,497,5458,125,133,Diffuse
995,чашка 7.jpg,4363,8414,2033,5309,164,125,Diffuse
996,чашка 7.jpg,4363,8414,1626,7268,121,113,Diffuse
997,чашка 7.jpg,4363,8414,1450,7334,160,90,Diffuse


In [8]:
val_df = df_from_json(dir_path=valid_data, json_filenames_list=list_of_json_files_val)
val_df

,filename,img_width,img_height,xmin,ymin,width,height,Type
0,чашка 12.jpg,4245,8498,584,763,382,334,Dense
1,чашка 12.jpg,4245,8498,1550,698,358,304,Dense
2,чашка 12.jpg,4245,8498,3285,817,256,346,Dense
3,чашка 12.jpg,4245,8498,2605,2069,209,280,Dense
4,чашка 12.jpg,4245,8498,1037,2409,238,388,Dense
...,...,...,...,...,...,...,...,...
290,чашка 13.jpg,4267,8520,935,7788,147,202,Mixed
291,чашка 13.jpg,4267,8520,269,7140,153,147,Mixed
292,чашка 13.jpg,4267,8520,3722,7228,108,171,Mixed
293,чашка 13.jpg,4267,8520,3668,6993,112,122,Diffuse


In [9]:
test_df = df_from_json(dir_path=test_data, json_filenames_list=list_of_json_files_test)
test_df

,filename,img_width,img_height,xmin,ymin,width,height,Type
0,чашка 8.1.jpg,4352,8498,538,1170,300,379,Dense
1,чашка 8.1.jpg,4352,8498,1470,751,348,356,Dense
2,чашка 8.1.jpg,4352,8498,2553,466,340,277,Dense
3,чашка 8.1.jpg,4352,8498,2000,498,182,166,Diffuse
4,чашка 8.1.jpg,4352,8498,3075,379,158,174,Diffuse
...,...,...,...,...,...,...,...,...
383,чашка 9.jpg,4325,8472,1520,5672,197,213,Mixed
384,чашка 9.jpg,4325,8472,1197,6074,205,189,Mixed
385,чашка 9.jpg,4325,8472,1316,6641,228,307,Diffuse
386,чашка 9.jpg,4325,8472,1410,6365,276,268,Mixed


Так как мы будем использовать модель YOLOv8 (выбор обоснован списком преимуществ), то необходимо переформатировать датасет из формата coco [x_min, y_min, width, hight] -> формат yolo normalized[x_center, y_center, width, height]

Список преимуществ YOLOv8:
 - Повышенная точность: YOLOv8 повышает точность обнаружения объектов по сравнению со своими предшественниками за счет внедрения новых методов и оптимизаций.
 - Повышенная скорость: YOLOv8 обеспечивает более высокую скорость вывода, чем другие модели обнаружения объектов, при сохранении высокой точности.
 - Несколько Backbones (основа архитектуры модели) : YOLOv8 поддерживает различные Backbones, такие как EfficientNet, ResNet и CSPDarknet, предоставляя пользователям гибкость в выборе наилучшей модели для их конкретного случая использования.
 - Адаптивное обучение: YOLOv8 использует адаптивное обучение для оптимизации скорости обучения и балансировки функции потерь во время обучения, что приводит к повышению производительности модели.
 - Продвинутое увеличение объема данных (Data Augmentation): YOLOv8 использует передовые методы Data Augmentation, такие как MixUp и CutMix, для повышения надежности и обобщения модели.
 - Настраиваемая архитектура: Архитектура YOLOv8 легко настраивается, позволяя пользователям легко изменять структуру и параметры модели в соответствии со своими потребностями.
 - Предварительно обученные модели: YOLOv8 предоставляет предварительно обученные модели для удобства использования и переноса обучения на различные наборы данных.

In [10]:
# Составление директорий под формат датасета YOLO
if not os.path.exists("..\\ML\\yolo_dataset\\images"): os.makedirs("..\\ML\\yolo_dataset\\images")

if not os.path.exists("..\\ML\\yolo_dataset\\images\\train"): os.makedirs("..\\ML\\yolo_dataset\\images\\train")
if not os.path.exists("..\\ML\\yolo_dataset\\images\\val"): os.makedirs("..\\ML\\yolo_dataset\\images\\val")
if not os.path.exists("..\\ML\\yolo_dataset\\images\\test"): os.makedirs("..\\ML\\yolo_dataset\\images\\test")

if not os.path.exists("..\\ML\\yolo_dataset\\labels"): os.makedirs("..\\ML\\yolo_dataset\\labels")

if not os.path.exists("..\\ML\\yolo_dataset\\labels\\train"): os.makedirs("..\\ML\\yolo_dataset\\labels\\train")
if not os.path.exists("..\\ML\\yolo_dataset\\labels\\val"): os.makedirs("..\\ML\\yolo_dataset\\labels\\val")
if not os.path.exists("..\\ML\\yolo_dataset\\labels\\test"): os.makedirs("..\\ML\\yolo_dataset\\labels\\test")

In [11]:
yolo_train_img = r'../ML/yolo_dataset/images/train/'
yolo_val_img = r'../ML/yolo_dataset/images/val/'
yolo_test_img = r'../ML/yolo_dataset/images/test/'

yolo_train_label = r'../ML/yolo_dataset/labels/train/'
yolo_val_label = r'../ML/yolo_dataset/labels/val/'
yolo_test_label = r'../ML/yolo_dataset/labels/test/'

In [12]:
def yolo_df_format(df: pd.DataFrame, class_dict: dict) -> pd.DataFrame:
    df_yolo = df.copy(deep=True)
    df_yolo["class"] = 0
    df_yolo.rename(columns={'filename':'img_name'}, inplace=True)
    
    for key, value in class_dict.items():
        df_yolo.loc[df['Type'] == value, 'class'] = key

    df_yolo["x_center"] = df_yolo["xmin"] + (df_yolo["width"]/2)
    df_yolo["y_center"] = df_yolo["ymin"] + (df_yolo["height"]/2)

    #нормализация координат
    df_yolo["x_center"] = df_yolo["x_center"] / df_yolo['img_width']
    df_yolo["y_center"] = df_yolo["y_center"] / df_yolo['img_height']
    df_yolo["width"] = df_yolo["width"] / df_yolo['img_width']
    df_yolo["height"] = df_yolo["height"] / df_yolo['img_height']

    df_yolo = df_yolo[["img_name","class","x_center","y_center","width","height"]]
    return df_yolo

In [13]:
train_df_yolo = yolo_df_format(df=train_df, class_dict=class_dict)
print(train_df_yolo['class'].value_counts())
train_df_yolo

class
2    566
1    292
0    141
Name: count, dtype: int64


,img_name,class,x_center,y_center,width,height
0,чашка 10.jpg,0,0.271807,0.096199,0.043964,0.028830
1,чашка 10.jpg,0,0.413119,0.072017,0.066062,0.017651
2,чашка 10.jpg,0,0.538032,0.039480,0.062340,0.025065
3,чашка 10.jpg,0,0.666667,0.046011,0.040475,0.019534
4,чашка 10.jpg,0,0.845662,0.122617,0.056990,0.039068
...,...,...,...,...,...,...
994,чашка 7.jpg,1,0.128237,0.656584,0.028650,0.015807
995,чашка 7.jpg,1,0.484758,0.638400,0.037589,0.014856
996,чашка 7.jpg,1,0.386546,0.870513,0.027733,0.013430
997,чашка 7.jpg,1,0.350676,0.876991,0.036672,0.010696


In [14]:
val_df_yolo = yolo_df_format(df=val_df, class_dict=class_dict)
print(val_df_yolo['class'].value_counts())
val_df_yolo

class
2    200
0     50
1     45
Name: count, dtype: int64


,img_name,class,x_center,y_center,width,height
0,чашка 12.jpg,0,0.182568,0.109438,0.089988,0.039303
1,чашка 12.jpg,0,0.407303,0.100024,0.084335,0.035773
2,чашка 12.jpg,0,0.804005,0.116498,0.060306,0.040715
3,чашка 12.jpg,0,0.638280,0.259944,0.049234,0.032949
4,чашка 12.jpg,0,0.272320,0.306307,0.056066,0.045658
...,...,...,...,...,...,...
290,чашка 13.jpg,2,0.236349,0.925939,0.034450,0.023709
291,чашка 13.jpg,2,0.080970,0.846655,0.035857,0.017254
292,чашка 13.jpg,2,0.884931,0.858392,0.025311,0.020070
293,чашка 13.jpg,1,0.872744,0.827934,0.026248,0.014319


In [15]:
test_df_yolo = yolo_df_format(df=test_df, class_dict=class_dict)
print(test_df_yolo['class'].value_counts())
test_df_yolo

class
2    181
0    156
1     51
Name: count, dtype: int64


,img_name,class,x_center,y_center,width,height
0,чашка 8.1.jpg,0,0.158088,0.159979,0.068934,0.044599
1,чашка 8.1.jpg,0,0.377757,0.109320,0.079963,0.041892
2,чашка 8.1.jpg,0,0.625689,0.071134,0.078125,0.032596
3,чашка 8.1.jpg,1,0.480469,0.068369,0.041820,0.019534
4,чашка 8.1.jpg,1,0.724724,0.054836,0.036305,0.020475
...,...,...,...,...,...,...
383,чашка 9.jpg,2,0.374220,0.682070,0.045549,0.025142
384,чашка 9.jpg,2,0.300462,0.728104,0.047399,0.022309
385,чашка 9.jpg,1,0.330636,0.801995,0.052717,0.036237
386,чашка 9.jpg,2,0.357919,0.767115,0.063815,0.031634


После того как подготовили датасет для формата yolo, необходимо составть txt файлы для соответствующих им filename.jpg. Пометка классов в них составляется форматом:

object-class x y width height

object-class - целочисленное значение от 0 до (кол-во классов-1)
x y width height - рациональные числа диапазона (0.0 to 1.0]
 - width height - ширина и высота bbox 
 - x y - центры прямоугольная bbox

In [16]:
train_img_list = sorted([img for img in os.listdir(train_data) if img.endswith('.jpg')])
train_img_list

['чашка 10.jpg',
 'чашка 11.jpg',
 'чашка 2.jpg',
 'чашка 3.jpg',
 'чашка 4.jpg',
 'чашка 5.jpg',
 'чашка 6.jpg',
 'чашка 7.jpg']

In [17]:
val_img_list = sorted([img for img in os.listdir(valid_data) if img.endswith('.jpg')])
val_img_list

['чашка 12.jpg', 'чашка 13.jpg']

In [18]:
test_img_list = sorted([img for img in os.listdir(test_data) if img.endswith('.jpg')])
test_img_list

['чашка 8.1.jpg', 'чашка 8.2.jpg', 'чашка 9.jpg']

In [19]:
def create_txt_labels(df: pd.DataFrame, img_dir: str, new_img_dir: str, label_dir: str) -> None:
    img_list = sorted([img for img in os.listdir(img_dir) if img.endswith('.jpg')])
    for i, img_name in enumerate(img_list):
        if np.isin(img_name, df['img_name']):
            name, extension = os.path.splitext(f'{img_name}')
            columns = ["class", "x_center", "y_center", "width", "height"]
            img_box = df[df['img_name'] == img_name][columns].values
            label_path = os.path.join(label_dir, name + ".txt")
            with open(label_path, "w+") as f:
                for row in img_box:
                    text = " ".join(str(x) for x in row[1:])
                    f.write(str(int(row[0]))+ ' ' + text)
                    f.write("\n")
    
        old_image_path = os.path.join(img_dir, img_name)
        new_image_path = os.path.join(new_img_dir, img_name)
        shutil.copy(old_image_path, new_image_path)
    return

In [20]:
create_txt_labels(df=train_df_yolo, img_dir=train_data, new_img_dir=yolo_train_img, label_dir=yolo_train_label)
create_txt_labels(df=val_df_yolo, img_dir=valid_data, new_img_dir=yolo_val_img, label_dir=yolo_val_label)
create_txt_labels(df=test_df_yolo, img_dir=test_data, new_img_dir=yolo_test_img, label_dir=yolo_test_label)

Подготовка к тренировке модели YOLOv8

In [21]:
%%writefile yolo.yaml
path: ../yolo_dataset
train: images/train
val: images/val
test: images/test

# Classes
names:
  0: Dense
  1: Diffuse
  2: Mixed

Overwriting yolo.yaml


In [22]:
!nvidia-smi

Tue Feb 27 12:47:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  |   00000000:01:00.0  On |                  N/A |
| 80%   32C    P0             51W /  220W |     793MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
ultralytics.checks()

Ultralytics YOLOv8.1.18 🚀 Python-3.11.6 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Setup complete ✅ (12 CPUs, 31.8 GB RAM, 420.7/931.5 GB disk)


In [24]:
#model = ultralytics.YOLO('yolov8m.pt')

#results = model.train(
#    data = 'yolo.yaml',
#    imgsz = resize_to,
#    epochs = num_epochs,
#    batch = batch_size,
#    name = 'yolov8m_50e'
#)

In [27]:
# Настройки для обучения модели и запуск процесса обучения
!yolo \
task = detect \
mode = train \
patience = 15 \
pretrained = True \
model = yolov8m.pt \
imgsz = {resize_to} \
data = yolo.yaml \
epochs = {num_epochs} \
batch = {batch_size} \
exist_ok = True \
plots = True \
name = yolov8m_{num_epochs}e

^C


In [37]:
# Настройки для валидации модели и запуск процесса валидации
!yolo \
task = detect \
mode = val \
model = runs/detect/yolov8m_50e/weights/best.pt \
data = yolo.yaml \
name = yolov8m_50e_eval \
plots = True

Ultralytics YOLOv8.1.18 рџљЂ Python-3.11.6 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs
                   all          2        295      0.635      0.668      0.653      0.303
                 Dense          2         50      0.919      0.451      0.769      0.431
               Diffuse          2         45      0.395      0.689      0.438      0.165
                 Mixed          2        200      0.592      0.865      0.752      0.312
Speed: 1.0ms preprocess, 83.5ms inference, 0.0ms loss, 42.4ms postprocess per image
Results saved to runs\detect\yolov8m_50e_eval
рџ’Ў Learn more at https://docs.ultralytics.com/modes/val



val: Scanning D:\Testovoe\ML\yolo_dataset\labels\val.cache... 2 images, 0 backgrounds, 0 corrupt: 100%|##########| 2/2 [00:00<?, ?it/s]
val: Scanning D:\Testovoe\ML\yolo_dataset\labels\val.cache... 2 images, 0 backgrounds, 0 corrupt: 100%|##########| 2/2 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|##########| 1/1 [00:04<00:00,  4.47s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|##########| 1/1 [00:04<00:00,  4.47s/it]


In [39]:
# Настройки для тестирования модели и запуск процесса тестирования
# Default настройка для IoU = 0.7
!yolo \
task = detect \
mode = predict \
model = runs/detect/yolov8m_50e/weights/best.pt \
source = yolo_dataset/images/test \
imgsz = 640 \
name = yolov8m_50e_pred \
save = True \
save_txt = True

Ultralytics YOLOv8.1.18 рџљЂ Python-3.11.6 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs

image 1/3 D:\Testovoe\ML\yolo_dataset\images\test\С‡Р°С€РєР° 8.1.jpg: 640x352 30 Denses, 53 Diffuses, 127 Mixeds, 238.9ms
image 2/3 D:\Testovoe\ML\yolo_dataset\images\test\С‡Р°С€РєР° 8.2.jpg: 640x352 34 Denses, 34 Diffuses, 208 Mixeds, 25.6ms
image 3/3 D:\Testovoe\ML\yolo_dataset\images\test\С‡Р°С€РєР° 9.jpg: 640x352 2 Denses, 78 Diffuses, 101 Mixeds, 10.0ms
Speed: 6.5ms preprocess, 91.5ms inference, 61.4ms postprocess per image at shape (1, 3, 640, 352)
Results saved to runs\detect\yolov8m_50e_pred2
3 labels saved to runs\detect\yolov8m_50e_pred2\labels
рџ’Ў Learn more at https://docs.ultralytics.com/modes/predict


Cчетчик объектов на каждом изображении

In [48]:
model = ultralytics.YOLO(r'runs/detect/yolov8m_50e/weights/best.pt')

res = model.predict(r'yolo_dataset/images/test')
classes_detected = {y: 0 for _, y in class_dict.items()}
for elem in res:
    names = elem.names
    filename = elem.path.split("\\")[-1]
    
    class_detections_values = []
    for k, v in names.items():
        class_detections_values.append(elem.boxes.cls.tolist().count(k))
    classes_detected_in_img = dict(zip(names.values(), class_detections_values))
    classes_detected = {k: classes_detected.get(k, 0) + classes_detected_in_img.get(k, 0) 
                        for k in set(classes_detected) & set(classes_detected_in_img)}
    print(f'Сумма объектов всех классов на изображении {filename} : {classes_detected_in_img}\n')
    
print(f'Сумма объектов всех классов на тестовых изображениях : {classes_detected}')


image 1/3 D:\Testovoe\ML\yolo_dataset\images\test\чашка 8.1.jpg: 640x352 30 Denses, 53 Diffuses, 127 Mixeds, 7.5ms
image 2/3 D:\Testovoe\ML\yolo_dataset\images\test\чашка 8.2.jpg: 640x352 34 Denses, 34 Diffuses, 208 Mixeds, 9.2ms
image 3/3 D:\Testovoe\ML\yolo_dataset\images\test\чашка 9.jpg: 640x352 2 Denses, 78 Diffuses, 101 Mixeds, 8.5ms
Speed: 1.7ms preprocess, 8.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 352)
Сумма объектов всех классов на изображении чашка 8.1.jpg : {'Dense': 30, 'Diffuse': 53, 'Mixed': 127}

Сумма объектов всех классов на изображении чашка 8.2.jpg : {'Dense': 34, 'Diffuse': 34, 'Mixed': 208}

Сумма объектов всех классов на изображении чашка 9.jpg : {'Dense': 2, 'Diffuse': 78, 'Mixed': 101}

Сумма объектов всех классов на тестовых изображениях : {'Dense': 66, 'Diffuse': 165, 'Mixed': 436}



Возможности по улучшению качества классификации модели:
 - Увеличение датасета: увеличить количество изображений (возможно путем наполнения датасета аугментациями изначальных данных)
 - При малом кол-ве данных можно использовать метод кросс-валидации на тренировочном датасете
 - Использовать более точную предтриноворочную модель YOLOv8x
 - По возможности за счет большого количества видеопамяти обучить и начальные слои
 - Перебором гиперпараметров обучения
 - Перебор классификаторов для нахождения лучшего результата